In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

pd.options.display.max_columns = None
pd.options.display.max_rows = 50

In [2]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

In [3]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}
status_name_dict = {0: "HC", 1: "LC_NEG", 2: "LC_POS"}

# Combine Results

In [35]:
import glob

path = "OUTPUT/MP/05-classifiers/DataV3"
files = glob.glob(path + '/*/CA*.csv')
# files = files+glob.glob(path + "/*/FS*.csv")
files = files + glob.glob(path + "/*/Val*.csv")

print(files)
print(len(files))

['OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/CA_dfcmplt_SITE2_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/CA_dfcmplt_SITE2_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE3/CA_dfcmplt_SITE3_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE3/CA_dfcmplt_SITE3_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt/CA_dfcmplt_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt/CA_dfcmplt_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE1/CA_dfcmplt_SITE1_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE1/CA_dfcmplt_SITE1_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE4/CA_dfcmplt_SITE4_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE4/CA_dfcmplt_SITE4_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC/CA_dfcmpltPreLC_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC/CA_dfcmpltPreLC_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/Val_dfcmplt_SITE2_HoldoutVal_FS_dfcmplt_Boruta_T1000_itrr500_t

In [36]:
ldf = []
for i in files:
    ldf.append(pd.read_csv(i))

In [37]:
dfRes = pd.concat(ldf)

In [38]:
dfRes

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_balanced_accuracy,split1_test_balanced_accuracy,split2_test_balanced_accuracy,split3_test_balanced_accuracy,split4_test_balanced_accuracy,mean_test_balanced_accuracy,std_test_balanced_accuracy,rank_test_balanced_accuracy,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_sensitivity,split1_test_sensitivity,split2_test_sensitivity,split3_test_sensitivity,split4_test_sensitivity,mean_test_sensitivity,std_test_sensitivity,rank_test_sensitivity,split0_test_specificity,split1_test_specificity,split2_test_specificity,split3_test_specificity,split4_test_specificity,mean_test_specificity,std_test_specificity,rank_test_specificity,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_NPV,split1_test_NPV,split2_test_NPV,split3_test_NPV,split4_test_NPV,mean_test_NPV,std_test_NPV,rank_test_NPV,split0_test_PPV,split1_test_PPV,split2_test_PPV,split3_test_PPV,split4_test_PPV,mean_test_PPV,std_test_PPV,rank_test_PPV,cv,Title,RunType,MainDataset,date,classifier,FeatureSet
0,0.090777,0.019057,0.019697,0.004308,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.846154,0.811966,0.862069,0.827586,0.801724,0.829900,0.022000,40.0,0.756757,0.702703,0.783784,0.722222,0.680556,0.729204,0.037029,40.0,0.947973,0.972635,0.969894,0.957292,0.956944,0.960948,0.009103,30.0,0.678571,0.576923,0.724138,0.615385,0.530612,0.625126,0.069298,40.0,0.513514,0.405405,0.567568,0.444444,0.361111,0.458408,0.074059,40.0,0.513514,0.405405,0.567568,0.444444,0.361111,0.458408,0.074059,40.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,0.816327,0.784314,0.831579,0.800000,0.776699,0.801784,0.020184,40.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,5,dfcmplt_SITE2_Holdout,Holdout,NaN,NaN,NaN,NaN
1,1.006718,0.124943,0.077636,0.014889,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.854701,0.820513,0.853448,0.793103,0.801724,0.824698,0.025575,41.0,0.770270,0.716216,0.770270,0.666667,0.680556,0.720796,0.043510,41.0,0.950000,0.972973,0.968525,0.955208,0.955208,0.960383,0.008788,32.0,0.701754,0.603774,0.701754,0.500000,0.530612,0.607579,0.083964,41.0,0.540541,0.432432,0.540541,0.333333,0.361111,0.441592,0.087020,41.0,0.540541,0.432432,0.540541,0.333333,0.361111,0.441592,0.087020,41.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,0.824742,0.792079,0.822917,0.769231,0.776699,0.797134,0.023016,41.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,5,dfcmplt_SITE2_Holdout,Holdout,NaN,NaN,NaN,NaN
2,0.097288,0.022931,0.019531,0.004972,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.923077,0.880342,0.905172,0.887931,0.879310,0.895167,0.016749,39.0,0.878378,0.810811,0.851351,0.819444,0.805556,0.833108,0.027659,39.0,0.984797,0.993243,0.989394,0.984028,0.977431,0.985779,0.005339,8.0,0.861538,0.766667,0.825397,0.779661,0.758621,0.798377,0.039130,39.0,0.756757,0.621622,0.702703,0.638

In [41]:
dfRes["classifier"] = dfRes["param_clf"].apply(lambda x: x.__str__().split("(")[0])
lColFirst = [
    "MainDataset",
    "Title",
    "date",
    # "classifier",
    'RunType',
    "cv",
    # "brt_nTrue",
    # "brt_nTop",
]
dfRes = MPutils.reorder_columns(dfRes, lColFirst)
dfRes.to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined.csv", index=False)
lColsSel = lColFirst + [
    x
    for x in dfRes
    if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
    and x not in lColFirst
]
dfRes.to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined_SelCol.csv")

# Import Data

In [5]:
dfRes

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV,brt_md,brt_params
0,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.850407,0.814844,0.946028,0.772186,0.635725,0.635725,0.993962,0.986031,0.803611,0.986031,NaN,T1000_itrr500_th100_topR5
1,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.843139,0.806170,0.951976,0.759603,0.619893,0.619893,0.992447,0.982396,0.796329,0.982396,NaN,T1000_itrr500_th100_topR5
2,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.882147,0.856343,0.970935,0.831323,0.726277,0.726277,0.986409,0.972767,0.843696,0.972767,NaN,T1000_itrr500_th100_topR5
3,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.884862,0.858963,0.973094,0.834447,0.728498,0.728498,0.989428,0.978568,0.845549,0.978568,NaN,T1000_itrr500_th100_topR5
4,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.911144,0.893260,0.979676,0.878492,0.803141,0.803141,0.983379,0.970617,0.882231,0.970617,NaN,T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5
1636,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5
1637,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5
1638,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5


In [6]:
dfRes.columns = [x.replace('mean_test_','') for x in dfRes.columns]

In [7]:
dfRes

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
0,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.850407,0.814844,0.946028,0.772186,0.635725,0.635725,0.993962,0.986031,0.803611,0.986031,NaN,T1000_itrr500_th100_topR5
1,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.843139,0.806170,0.951976,0.759603,0.619893,0.619893,0.992447,0.982396,0.796329,0.982396,NaN,T1000_itrr500_th100_topR5
2,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.882147,0.856343,0.970935,0.831323,0.726277,0.726277,0.986409,0.972767,0.843696,0.972767,NaN,T1000_itrr500_th100_topR5
3,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.884862,0.858963,0.973094,0.834447,0.728498,0.728498,0.989428,0.978568,0.845549,0.978568,NaN,T1000_itrr500_th100_topR5
4,dfcmplt,dfcmplt_Holdout,2024-04-30 14:42:05.089744,RandomForestClassifier,Holdout,5,NaN,NaN,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.911144,0.893260,0.979676,0.878492,0.803141,0.803141,0.983379,0.970617,0.882231,0.970617,NaN,T1000_itrr500_th100_topR5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5
1636,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5
1637,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5
1638,dfcmplt_SITE4,dfcmplt_SITE4_Boruta_Top,2024-04-30 16:20:27.877332,XGBClassifier,Boruta_Top,5,46.0,50.0,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.974597,0.958974,0.998462,0.984593,0.984615,0.984615,0.933333,0.985714,0.942857,0.985714,7.0,T1000_itrr500_th100_topR5


In [8]:
s_brt = 'and (RunType=="Boruta_True" or RunType=="Boruta_Top")'
s_RF10 = 'classifier=="RandomForestClassifier" and param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_RFXGB10 = 'param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_CMPLT = 'MainDataset=="dfcmplt"'

In [19]:
lPrettyCols =     [
        "MainDataset",
        "RunType",
        "classifier",
        "brt_nTrue",
        "brt_nTop",
        "accuracy",
        "balanced_accuracy",
        "roc_auc",
        "f1",
        "recall",
        "sensitivity",
        "specificity",
        "precision",
        "NPV",
        "PPV",
        "brt_md",
        'brt_params'
    ]
lPrettyColsSite = [
    "MainDataset",
    "RunType",
    'Site',
    "classifier",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
]

# Inspect Original Holdout
- Summary - Across all the sites, it looks like the holdout set can naturally classify well

In [10]:
dfRes.query(f'{s_RF10} and RunType=="Holdout"').sort_values("balanced_accuracy", ascending=False)[lPrettyCols].T

,1317,989,333,5,661
MainDataset,dfcmplt_SITE4,dfcmplt_SITE3,dfcmplt_SITE1,dfcmplt,dfcmplt_SITE2
RunType,Holdout,Holdout,Holdout,Holdout,Holdout
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
brt_nTrue,NaN,NaN,NaN,NaN,NaN
brt_nTop,NaN,NaN,NaN,NaN,NaN
accuracy,1.0,0.985714,0.94697,0.912049,0.927822
balanced_accuracy,1.0,0.990909,0.957143,0.894043,0.88506
roc_auc,1.0,1.0,0.964286,0.982516,0.995042
f1,1.0,0.990476,0.941818,0.879569,0.869687
recall,1.0,0.981818,1.0,0.803192,0.77012


In [11]:
s_CMPLT

'MainDataset=="dfcmplt"'

In [20]:
s_query = f'{s_CMPLT} and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values('balanced_accuracy',ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="dfcmplt" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.9468024632572238


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
251,dfcmplt,Boruta_True,RandomForestClassifier,68.0,71.0,0.951979,0.946802,0.985201,0.938600,0.920787,0.920787,0.972818,0.957849,0.948886,0.957849,7.0,T1000_itrr500_th100_topR5
169,dfcmplt,Boruta_True,RandomForestClassifier,81.0,85.0,0.951074,0.945679,0.986266,0.937359,0.918539,0.918539,0.972818,0.957883,0.947572,0.957883,5.0,T1000_itrr500_th100_topR5
292,dfcmplt,Boruta_Top,RandomForestClassifier,68.0,71.0,0.951074,0.945679,0.985995,0.937262,0.918539,0.918539,0.972818,0.957817,0.947696,0.957817,7.0,T1000_itrr500_th100_topR5
210,dfcmplt,Boruta_Top,RandomForestClassifier,81.0,85.0,0.950169,0.944543,0.986511,0.936161,0.916267,0.916267,0.972818,0.957837,0.946140,0.957837,5.0,T1000_itrr500_th100_topR5
221,dfcmplt,Boruta_Top,XGBClassifier,81.0,85.0,0.944731,0.938873,0.969870,0.928972,0.909474,0.909474,0.968273,0.950551,0.942047,0.950551,5.0,T1000_itrr500_th100_topR5
139,dfcmplt,Boruta_Top,XGBClassifier,113.0,117.0,0.944735,0.937768,0.969536,0.928712,0.902707,0.902707,0.972830,0.957116,0.937849,0.957116,3.0,T1000_itrr500_th100_topR5
303,dfcmplt,Boruta_Top,XGBClassifier,68.0,71.0,0.942020,0.936215,0.970015,0.926033,0.907176,0.907176,0.965254,0.946008,0.939828,0.946008,7.0,T1000_itrr500_th100_topR5
87,dfcmplt,Boruta_True,RandomForestClassifier,113.0,117.0,0.943821,0.935491,0.987560,0.926839,0.893641,0.893641,0.977341,0.963409,0.932760,0.963409,3.0,T1000_itrr500_th100_topR5
180,dfcmplt,Boruta_True,XGBClassifier,81.0,85.0,0.941107,0.935098,0.970057,0.924760,0.904954,0.904954,0.965243,0.945818,0.938489,0.945818,5.0,T1000_itrr500_th100_topR5
190,dfcmplt,Boruta_True,XGBClassifier,81.0,85.0,0.942012,0.934727,0.968622,0.925266,0.898161,0.898161,0.971292,0.954583,0.934822,0.954583,5.0,T1000_itrr500_th100_topR5


In [14]:
# dfTmp.query('classifier=="RandomForestClassifier" and (RunType=="Boruta_True" or RunType=="Boruta_Top")')
dfTmp.query(
    'classifier=="RandomForestClassifier"'
)

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
251,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,RandomForestClassifier,Boruta_True,5,68.0,71.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.951979,0.946802,0.985201,0.938600,0.920787,0.920787,0.972818,0.957849,0.948886,0.957849,7.0,T1000_itrr500_th100_topR5
169,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,RandomForestClassifier,Boruta_True,5,81.0,85.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.951074,0.945679,0.986266,0.937359,0.918539,0.918539,0.972818,0.957883,0.947572,0.957883,5.0,T1000_itrr500_th100_topR5
292,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,RandomForestClassifier,Boruta_Top,5,68.0,71.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.951074,0.945679,0.985995,0.937262,0.918539,0.918539,0.972818,0.957817,0.947696,0.957817,7.0,T1000_itrr500_th100_topR5
210,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,RandomForestClassifier,Boruta_Top,5,81.0,85.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.950169,0.944543,0.986511,0.936161,0.916267,0.916267,0.972818,0.957837,0.946140,0.957837,5.0,T1000_itrr500_th100_topR5
87,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,RandomForestClassifier,Boruta_True,5,113.0,117.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.943821,0.935491,0.987560,0.926839,0.893641,0.893641,0.977341,0.963409,0.932760,0.963409,3.0,T1000_itrr500_th100_topR5
128,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,RandomForestClassifier,Boruta_Top,5,113.0,117.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.942916,0.934368,0.987166,0.925630,0.891394,0.891394,0.977341,0.963353,0.931364,0.963353,3.0,T1000_itrr500_th100_topR5


In [15]:
dfTmp.query('classifier=="XGBClassifier"')

,MainDataset,Title,date,classifier,RunType,cv,brt_nTrue,brt_nTop,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
221,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_Top,5,81.0,85.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.944731,0.938873,0.969870,0.928972,0.909474,0.909474,0.968273,0.950551,0.942047,0.950551,5.0,T1000_itrr500_th100_topR5
139,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_Top,5,113.0,117.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.944735,0.937768,0.969536,0.928712,0.902707,0.902707,0.972830,0.957116,0.937849,0.957116,3.0,T1000_itrr500_th100_topR5
303,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_Top,5,68.0,71.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.942020,0.936215,0.970015,0.926033,0.907176,0.907176,0.965254,0.946008,0.939828,0.946008,7.0,T1000_itrr500_th100_topR5
180,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_True,5,81.0,85.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.941107,0.935098,0.970057,0.924760,0.904954,0.904954,0.965243,0.945818,0.938489,0.945818,5.0,T1000_itrr500_th100_topR5
190,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_True,5,81.0,85.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.942012,0.934727,0.968622,0.925266,0.898161,0.898161,0.971292,0.954583,0.934822,0.954583,5.0,T1000_itrr500_th100_topR5
108,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_True,5,113.0,117.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.940197,0.932858,0.969366,0.922845,0.895940,0.895940,0.969777,0.952358,0.933555,0.952358,3.0,T1000_itrr500_th100_topR5
262,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_True,5,68.0,71.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.939292,0.932850,0.973441,0.922168,0.900434,0.900434,0.965265,0.945549,0.935862,0.945549,7.0,T1000_itrr500_th100_topR5
98,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_True,5,113.0,117.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.939301,0.932125,0.968847,0.921703,0.895965,0.895965,0.968284,0.949593,0.933444,0.949593,3.0,T1000_itrr500_th100_topR5
149,dfcmplt,dfcmplt_Boruta_Top,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_Top,5,113.0,117.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.939301,0.932118,0.970289,0.921714,0.895940,0.895940,0.968296,0.950004,0.933529,0.950004,3.0,T1000_itrr500_th100_topR5
272,dfcmplt,dfcmplt_Boruta_True,2024-04-30 14:42:05.089744,XGBClassifier,Boruta_True,5,68.0,71.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.936578,0.929852,0.973117,0.918776,0.895965,0.895965,0.963739,0.943272,0.932895,0.943272,7.0,T1000_itrr500_th100_topR5


# Difference between Boruta Selected Datasets

In [21]:
lbrt_7 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
)

lbrt_5 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)

lbrt_3 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD3.csv"
)

len(lbrt_7), len(lbrt_5), len(lbrt_3)

(68, 81, 113)

In [24]:
set(lbrt_7).difference(set(lbrt_5))

set()

In [27]:
set(lbrt_5).difference(set(lbrt_7))

{'CD-covid_test_result_2',
 'CD-gcs',
 'CD-neurologic-other-1',
 'CD-neurologic-other-4',
 'CD-required_oxygen',
 'CD-respiratory_rate',
 'CD-severity_level___2',
 'LABS-IND-basophil_count',
 'LABS-IND-eosinophil',
 'LABS-IND-pct',
 'SYMPT-fever_38_0oc___2',
 'SYMPT-night_sweats___2',
 'SYMPT-runny_nose_rhinorrhea___2'}

In [32]:
set(lbrt_7).difference(set(lbrt_3))

set()

In [33]:
set(lbrt_3).difference(set(lbrt_5))

{'CD-covid_test_result_1',
 'CD-covid_test_result_3',
 'CD-hospitalized',
 'CD-insomnia___1',
 'CD-medication_received___4',
 'CD-medication_received___5',
 'CD-memory_problems___1',
 'CD-respiratory-other-1',
 'CD-severity_level___3',
 'CD-type_of_care___2',
 'HHX-Arterial Hypertension',
 'HHX-Diabetes',
 'HHX-OTHER-Sleep-disorder',
 'HHX-Obesity',
 'IMG-ctt',
 'LAB-test-lung_function',
 'LABS-IND-cpk',
 'LABS-IND-inr',
 'MED-Cardiovascular-agents',
 'MED-Respiratory-tract-agents',
 'SYMPT-cough___1',
 'SYMPT-diarrhea___1',
 'SYMPT-extremity_weakness_or_numb___1',
 'SYMPT-fatigue___1',
 'SYMPT-fever_38_0oc___1',
 'SYMPT-joint_pain_arthralgia___1',
 'SYMPT-muscle_aches_myalgia___1',
 'SYMPT-nausea_vomiting___1',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-sore_throat___1',
 'SYMPT-trouble_speaking_aphasia_d___2',
 'SYMPT-weight_loss___1'}

In [34]:
set(lbrt_3).difference(set(lbrt_5))

{'CD-covid_test_result_1',
 'CD-covid_test_result_3',
 'CD-hospitalized',
 'CD-insomnia___1',
 'CD-medication_received___4',
 'CD-medication_received___5',
 'CD-memory_problems___1',
 'CD-respiratory-other-1',
 'CD-severity_level___3',
 'CD-type_of_care___2',
 'HHX-Arterial Hypertension',
 'HHX-Diabetes',
 'HHX-OTHER-Sleep-disorder',
 'HHX-Obesity',
 'IMG-ctt',
 'LAB-test-lung_function',
 'LABS-IND-cpk',
 'LABS-IND-inr',
 'MED-Cardiovascular-agents',
 'MED-Respiratory-tract-agents',
 'SYMPT-cough___1',
 'SYMPT-diarrhea___1',
 'SYMPT-extremity_weakness_or_numb___1',
 'SYMPT-fatigue___1',
 'SYMPT-fever_38_0oc___1',
 'SYMPT-joint_pain_arthralgia___1',
 'SYMPT-muscle_aches_myalgia___1',
 'SYMPT-nausea_vomiting___1',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-sore_throat___1',
 'SYMPT-trouble_speaking_aphasia_d___2',
 'SYMPT-weight_loss___1'}

In [35]:
set(lbrt_5).difference(set(lbrt_3))

set()

In [30]:
set(lbrt_5).difference(set(lbrt_7))

{'CD-covid_test_result_2',
 'CD-gcs',
 'CD-neurologic-other-1',
 'CD-neurologic-other-4',
 'CD-required_oxygen',
 'CD-respiratory_rate',
 'CD-severity_level___2',
 'LABS-IND-basophil_count',
 'LABS-IND-eosinophil',
 'LABS-IND-pct',
 'SYMPT-fever_38_0oc___2',
 'SYMPT-night_sweats___2',
 'SYMPT-runny_nose_rhinorrhea___2'}

# Validation Result

- lCmplt = MPutils.getTrueFeatList("OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv")


- lColFS = lCmplt
- selFeatName = "FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7"

In [36]:
dfValRes = pd.read_csv(
    "OUTPUT/MP/05-classifiers/DataV3/CLFHoldoutValRunCombined_FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7_selCol.csv"
)
dfValRes.columns = [x.replace("mean_test_", "") for x in dfValRes.columns]

In [37]:
dfValRes

,MainDataset,Title,date,classifier,RunType,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV
0,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.884854,0.858958,0.963452,0.835139,0.728498,0.728498,0.989417,0.979101,0.845081,0.979101
1,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.883945,0.857808,0.964534,0.833637,0.726200,0.726200,0.989417,0.979183,0.843980,0.979183
2,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.921107,0.906113,0.978601,0.893591,0.830363,0.830363,0.981864,0.969058,0.896988,0.969058
3,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.913875,0.897112,0.979010,0.882865,0.812360,0.812360,0.981864,0.968757,0.887128,0.968757
4,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.940156,0.931743,0.982863,0.922105,0.889198,0.889198,0.974288,0.958830,0.929950,0.958830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.948996,0.938437,0.973932,0.927617,0.899246,0.899246,0.977628,0.959180,0.944480,0.959180
324,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.954670,0.943794,0.978004,0.935657,0.903243,0.903243,0.984345,0.971324,0.946611,0.971324
325,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.956078,0.946516,0.971732,0.937991,0.910935,0.910935,0.982097,0.967575,0.950608,0.967575
326,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.948996,0.937675,0.977302,0.927666,0.895475,0.895475,0.979875,0.962652,0.942180,0.962652


In [41]:
site_name_dict

{1: 'LDN', 2: 'MTL', 3: 'SAN', 4: 'RIO', 5: 'LUS', 6: 'CA', 7: 'NA'}

In [45]:
siteMap = {
    "dfcmplt": "Global",
    "dfcmplt_SITE1": "LDN",
    "dfcmplt_SITE2": "MTL",
    "dfcmplt_SITE3": "SAN",
    "dfcmplt_SITE4": "RIO",
    "dfcmplt_SITE5": "LUS",
    "dfcmplt_SITE6": "Canada",
    "dfcmplt_SITE7": "NorthAmerica",
}

In [47]:
dfValRes['MainDataset'] = dfValRes['MainDataset'].map(siteMap)

# Validation Results -- Final?

In [49]:
dfValRes.query(f'classifier=="RandomForestClassifier" and param_clf__max_depth==10 and param_clf__n_estimators==1000')[[x for x in lPrettyCols if 'brt_' not in x]].T

,5,46,87,128,169,210,251,292
MainDataset,Global,LDN,MTL,SAN,RIO,LUS,Canada,NorthAmerica
RunType,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
accuracy,0.94197,0.912121,0.979369,0.969231,0.993548,0.829932,0.967151,0.964589
balanced_accuracy,0.933624,0.912143,0.967117,0.980909,0.98,0.66486,0.956518,0.958143
roc_auc,0.983206,0.951429,0.994132,1.0,1.0,0.861153,0.98919,0.990181
f1,0.924669,0.892929,0.965854,0.97995,0.996226,0.480784,0.947304,0.95083
recall,0.891445,0.91,0.934234,0.961818,1.0,0.367273,0.926829,0.934163
sensitivity,0.891445,0.91,0.934234,0.961818,1.0,0.367273,0.926829,0.934163
specificity,0.975803,0.914286,1.0,1.0,0.96,0.962447,0.986207,0.982122


# Feature Ranking Results

In [51]:
dfFR = pd.read_csv('OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FR_dfcmplt_HoldoutValFS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv')

In [52]:
dfFR

,Rank,Feature,Importance
0,1,SYMPT-fatigue___2,0.1659
1,2,SYMPT-shortness_of_breath_dyspne___2,0.1153
2,3,SYMPT-loss_of_taste_lost_of_smel___2,0.0566
3,4,SYMPT-cough___2,0.0501
4,5,LABS-IND-triglycerides,0.0466
...,...,...,...
63,64,LABS-IND-basophil,0.0011
64,65,SYMPT-wheezing_or_stridor___2,0.0010
65,66,SYMPT-ear_pain___2,0.0009
66,66,CD-adjunctive___1,0.0009


In [57]:
dfFR[dfFR['Importance']>0.01]

,Rank,Feature,Importance
0,1,SYMPT-fatigue___2,0.1659
1,2,SYMPT-shortness_of_breath_dyspne___2,0.1153
2,3,SYMPT-loss_of_taste_lost_of_smel___2,0.0566
3,4,SYMPT-cough___2,0.0501
4,5,LABS-IND-triglycerides,0.0466
5,6,SYMPT-joint_pain_arthralgia___2,0.0444
6,7,SYMPT-muscle_aches_myalgia___2,0.0432
7,8,LABS-IND-monocyte_count,0.0410
8,9,SYMPT-headache___2,0.0384
9,10,LABS-IND-bilirubin,0.0356
